In [7]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np

%load_ext autoreload
%autoreload 2

cols = ['Record Id',	
'Created Time',
'Created Date',	
'First Name',	
'Last Name',	
'Email',	
'Phone',	
'Mobile',	
'Birthdate',	
'Year of Birth',
'Street',	
'Street2',	
'City',	
'State',	
'Country',	
'Nationality',
'Booking Status', 
'BookRef',
'Departure Date',	
'Arrival Date',	
'Card Type',	
'Marital Status',	
'Lead Brand',	
'Lead Sub-Brand',	
'Lead Source',	
'Lead Locations',	
'Lead Source Description',	
'Lead Regions',
'Lead Status',
'Email Opt Out',	
'Opt In',
'GDPR Compliant',
'Website',
'Comments',
'Do you live in Bali?']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
path = r'C:\Users\fajar.fatoni\Documents\Python\Data\Leads_2025_06_16.csv'
zoho_df = pd.read_csv(path, dtype='string')
# zoho_df = pd.read_excel(path, dtype='string', sheet_name='KSH')

zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin
zoho_df.fillna('empty', inplace=True)

zoho_df['Mobile'] = 'empty'
# zoho_df['Phone'] = 'empty'
zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Enquiry sent to,Mobile
0,zcrm_3584690000000459415,2019-01-09 11:11:09,empty,Graham,Bovet-White,grahambovet@gmail.com,+447831252500,United Kingdom,Karma Resorts,Other,empty,Past Guests,Karma St. Martin's;Karma La Herriza,empty,true,false,true,empty,empty,empty
1,zcrm_3584690000369606560,2022-05-24 12:39:51,empty,Rowan,Johnston,rowan.johnston22962@gmail.com,+61427997077,Australia,Karma Resorts,Other,empty,Past Guests,Karma Rottnest;Karma La Herriza,empty,false,false,false,Email And Phone,empty,empty
2,zcrm_3584690000382360020,2022-07-09 22:46:42,2022-07-09,Sijo,Eapen,sijoeapen@gmail.com,+919892718274,India,Karma Resorts,Other,Karma Group Booking Request Form,Other,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,false,empty,info@karmagroup.com,empty
3,zcrm_3584690000403535119,2022-10-06 23:33:18,2022-10-06,Josh,Lowsley-Williams,jlowsleywilliams@gmail.com,447779354907,Spain,Karma Resorts,Other,Karma Group Booking Request Form,Other,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,true,empty,info@karmagroup.com,empty
4,zcrm_3584690000409631627,2022-11-09 18:16:39,empty,Farinaz Neshati,Sani,farinesh@gmail.com,+4434928062444,United Kingdom,Karma Resorts,Other,Viewpoint,Past Guests,Karma Royal Candidasa;Karma La Herriza,empty,true,false,true,Email And Phone,empty,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,zcrm_3584690000556728404,2025-06-16 12:33:14,empty,Yanique,-,yaniquesparks@gmail.com,447702059275,United Kingdom,Karma Resorts,Other,Karma La Herriza Database (JUNE 2025) - Weddin...,Web Enquiry,Karma La Herriza,empty,true,true,true,Email and Phone,empty,empty
1122,zcrm_3584690000556728405,2025-06-16 12:33:14,empty,Charlotte,Harris,charlotteharris9@live.co.uk,447979505829,United Kingdom,Karma Resorts,Other,Karma La Herriza Database (JUNE 2025) - Weddin...,Web Enquiry,Karma La Herriza,empty,true,true,true,Email and Phone,empty,empty
1123,zcrm_3584690000556967179,2025-06-15 20:13:24,2025-06-15,empty,Lennart Schmidt,lennart.schmidt@ansys.com,+491735211584,Germany,Karma Resorts,Other,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,true,Email and Phone,events@karmalakeofmenteith.com,empty
1124,zcrm_3584690000557133839,2025-06-16 12:15:20,empty,Joseph,Galindo,galindo.joe@gmail.com,+17196614592,USA,Karma Resorts,Other,Viewpoint,Past Guests,Karma La Herriza,empty,true,false,false,Email and Phone,empty,empty


In [9]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


# ====================== Awal Filter Umum ====================== 

In [10]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')
m.clean_not_valid_email_format(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')
m.clean_not_valid_email_format(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')
m.clean_not_valid_email_format(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:105: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

'Email'

In [11]:
count_deleted_wrong_format = m.delete_email_wrong_format(zoho_df, 'Email')
count_after_deleted_wrong_format = zoho_df.shape[0]

# ====================== Akhir Filter Umum ====================== 

In [12]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])]

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Enquiry sent to,Mobile
10,zcrm_3584690000443277002,2023-05-05 05:13:51,empty,Michael,Bartolozzi,rbs73@hotmail.com,393483679930,Other,Karma Resorts,Other,Karma Group Web Online Booking,Web Enquiry,Karma La Herriza,https://karmagroup.com/booking-engine/,true,true,false,empty,reception@karmalaherriza.com,empty
22,zcrm_3584690000453643428,2023-07-17 15:24:55,empty,Brian,Hagan,bjphagan@gmail.com,447902941998,United Kingdom,Karma Resorts,Other,Viewpoint,Past Guests,Karma La Herriza,empty,true,false,true,Email And Phone,empty,empty
38,zcrm_3584690000457526852,2023-08-14 14:28:14,empty,Michael,Bartolozzi,rbs73@hotmail.com,393483679930,Italy,Karma Resorts,Other,Viewpoint,Past Guests,Karma La Herriza,empty,true,false,true,Email And Phone,empty,empty
87,zcrm_3584690000467468707,2023-10-30 15:16:55,empty,Thomas,Eldered,thomas@eldered.net,46708676210,Sweden,Karma Resorts,Other,Viewpoint,Past Guests,Karma La Herriza,empty,true,false,true,Email and Phone,empty,empty
119,zcrm_3584690000485951147,2024-03-01 16:35:15,2024-03-01,Junique,Wepener,juniquew@gmail.com,85587851773,Cambodia,Karma Resorts,Other,Karma Group Booking Request Form,Web Enquiry,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,false,Email and Phone,reception@karmalaherriza.com,empty
279,zcrm_3584690000498960011,2024-06-14 21:43:04,2024-06-14,Georgia,Luckhurst Mccord,georgiahrluckhurstmccord@yahoo.com,447922821674,United Kingdom,Karma Resorts,Other,Karma La Herriza - Weddings at Karma La Herriza,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,true,Email and Phone,empty,empty
297,zcrm_3584690000499902748,2024-06-24 15:28:31,empty,Francesca,Picciola,francescarose44@hotmail.co.uk,447885583677,United Kingdom,Karma Resorts,Other,Viewpoint,Past Guests,Karma La Herriza,empty,true,false,true,Email and Phone,empty,empty
358,zcrm_3584690000501758001,2024-07-09 23:35:42,empty,Rebecka,Sandin,sandinrebecka@gmail.com,46707427771,Sweden,Timeshare Marketing,Other,Karma La Herriza - Weddings at Karma La Herriza,Web Enquiry,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,true,Email and Phone,weddings@karmalaherriza.com,empty
362,zcrm_3584690000501845118,2024-07-10 19:52:44,2024-07-10,Joachik,Miller,joachim.edward1@gmail.com,447496608948,United Kingdom,Karma Resorts,Other,Karma Group Booking Request Form,Web Enquiry,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,true,Email and Phone,reception@karmalaherriza.com,empty
366,zcrm_3584690000501919237,2024-07-12 06:17:57,2024-07-12,Richard,Stokes,richardjstokes@gmail.com,447713199942,Spain,Karma Resorts,Other,Karma Group Booking Request Form,Web Enquiry,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,true,Email and Phone,reception@karmalaherriza.com,empty


In [13]:
zoho_df.drop_duplicates(subset=['Phone','Mobile','Email'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Enquiry sent to,Mobile
0,zcrm_3584690000000459415,2019-01-09 11:11:09,empty,Graham,Bovet-White,grahambovet@gmail.com,447831252500,United Kingdom,Karma Resorts,Other,empty,Past Guests,Karma St. Martin's;Karma La Herriza,empty,true,false,true,empty,empty,empty
1,zcrm_3584690000369606560,2022-05-24 12:39:51,empty,Rowan,Johnston,rowan.johnston22962@gmail.com,61427997077,Australia,Karma Resorts,Other,empty,Past Guests,Karma Rottnest;Karma La Herriza,empty,false,false,false,Email And Phone,empty,empty
2,zcrm_3584690000382360020,2022-07-09 22:46:42,2022-07-09,Sijo,Eapen,sijoeapen@gmail.com,919892718274,India,Karma Resorts,Other,Karma Group Booking Request Form,Other,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,false,empty,info@karmagroup.com,empty
3,zcrm_3584690000403535119,2022-10-06 23:33:18,2022-10-06,Josh,Lowsley-Williams,jlowsleywilliams@gmail.com,447779354907,Spain,Karma Resorts,Other,Karma Group Booking Request Form,Other,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,true,empty,info@karmagroup.com,empty
4,zcrm_3584690000409631627,2022-11-09 18:16:39,empty,Farinaz Neshati,Sani,farinesh@gmail.com,4434928062444,United Kingdom,Karma Resorts,Other,Viewpoint,Past Guests,Karma Royal Candidasa;Karma La Herriza,empty,true,false,true,Email And Phone,empty,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,zcrm_3584690000556728404,2025-06-16 12:33:14,empty,Yanique,-,yaniquesparks@gmail.com,447702059275,United Kingdom,Karma Resorts,Other,Karma La Herriza Database (JUNE 2025) - Weddin...,Web Enquiry,Karma La Herriza,empty,true,true,true,Email and Phone,empty,empty
1100,zcrm_3584690000556728405,2025-06-16 12:33:14,empty,Charlotte,Harris,charlotteharris9@live.co.uk,447979505829,United Kingdom,Karma Resorts,Other,Karma La Herriza Database (JUNE 2025) - Weddin...,Web Enquiry,Karma La Herriza,empty,true,true,true,Email and Phone,empty,empty
1101,zcrm_3584690000556967179,2025-06-15 20:13:24,2025-06-15,empty,Lennart Schmidt,lennart.schmidt@ansys.com,491735211584,Germany,Karma Resorts,Other,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,true,Email and Phone,events@karmalakeofmenteith.com,empty
1102,zcrm_3584690000557133839,2025-06-16 12:15:20,empty,Joseph,Galindo,galindo.joe@gmail.com,17196614592,USA,Karma Resorts,Other,Viewpoint,Past Guests,Karma La Herriza,empty,true,false,false,Email and Phone,empty,empty


In [14]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test$', flags=re.I, regex=True)
    ]

count_leads_test = test_leads.shape[0]

# test_leads.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\test_leads_2.xlsx', index=False)
test_leads

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Enquiry sent to,Mobile


In [15]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [16]:
# dnm_leads = zoho_df.loc[(zoho_df['Do Not Mail'] == 'true') | (zoho_df['Do Not Contact'] == 'true')]
# count_dnm = dnm_leads.shape[0]

In [17]:
# zoho_df.drop(dnm_leads.index, inplace=True)
# zoho_df.reset_index(drop=True, inplace=True)

# count_after_dedup_dnm = zoho_df.shape[0]

In [18]:
# Cek Email OTA
email_OTA = m.find_OTA(zoho_df, 'Email')
count_email_OTA = email_OTA.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_OTA

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Enquiry sent to,Mobile


In [19]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Enquiry sent to,Mobile
0,zcrm_3584690000000459415,2019-01-09 11:11:09,empty,Graham,Bovet-White,grahambovet@gmail.com,447831252500,United Kingdom,Karma Resorts,Other,empty,Past Guests,Karma St. Martin's;Karma La Herriza,empty,true,false,true,empty,empty,empty
1,zcrm_3584690000369606560,2022-05-24 12:39:51,empty,Rowan,Johnston,rowan.johnston22962@gmail.com,61427997077,Australia,Karma Resorts,Other,empty,Past Guests,Karma Rottnest;Karma La Herriza,empty,false,false,false,Email And Phone,empty,empty
2,zcrm_3584690000382360020,2022-07-09 22:46:42,2022-07-09,Sijo,Eapen,sijoeapen@gmail.com,919892718274,India,Karma Resorts,Other,Karma Group Booking Request Form,Other,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,false,empty,info@karmagroup.com,empty
3,zcrm_3584690000403535119,2022-10-06 23:33:18,2022-10-06,Josh,Lowsley-Williams,jlowsleywilliams@gmail.com,447779354907,Spain,Karma Resorts,Other,Karma Group Booking Request Form,Other,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,true,empty,info@karmagroup.com,empty
4,zcrm_3584690000409631627,2022-11-09 18:16:39,empty,Farinaz Neshati,Sani,farinesh@gmail.com,4434928062444,United Kingdom,Karma Resorts,Other,Viewpoint,Past Guests,Karma Royal Candidasa;Karma La Herriza,empty,true,false,true,Email And Phone,empty,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,zcrm_3584690000556728404,2025-06-16 12:33:14,empty,Yanique,-,yaniquesparks@gmail.com,447702059275,United Kingdom,Karma Resorts,Other,Karma La Herriza Database (JUNE 2025) - Weddin...,Web Enquiry,Karma La Herriza,empty,true,true,true,Email and Phone,empty,empty
1100,zcrm_3584690000556728405,2025-06-16 12:33:14,empty,Charlotte,Harris,charlotteharris9@live.co.uk,447979505829,United Kingdom,Karma Resorts,Other,Karma La Herriza Database (JUNE 2025) - Weddin...,Web Enquiry,Karma La Herriza,empty,true,true,true,Email and Phone,empty,empty
1101,zcrm_3584690000556967179,2025-06-15 20:13:24,2025-06-15,empty,Lennart Schmidt,lennart.schmidt@ansys.com,491735211584,Germany,Karma Resorts,Other,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,true,Email and Phone,events@karmalakeofmenteith.com,empty
1102,zcrm_3584690000557133839,2025-06-16 12:15:20,empty,Joseph,Galindo,galindo.joe@gmail.com,17196614592,USA,Karma Resorts,Other,Viewpoint,Past Guests,Karma La Herriza,empty,true,false,false,Email and Phone,empty,empty


In [20]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Enquiry sent to,Mobile
104,zcrm_3584690000484131001,2024-02-16 23:53:58,2024-02-16,empty,Adrian Willmers,gm@karmasalfordhall.com,447827542105,United Kingdom,Karma Resorts,Other,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,true,Email and Phone,events@karmalakeofmenteith.com,empty
112,zcrm_3584690000485596088,2024-02-29 15:44:05,2024-02-29,empty,Shivani,shivani.yeshwant@karmagroup.com,917682075005,India,Karma Resorts,Other,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,false,Email and Phone,events@karmalakeofmenteith.com,empty
541,zcrm_3584690000508307043,2024-09-17 19:56:20,2024-09-17,empty,James,headchef@karmasalfordhall.com,447412663350,United Kingdom,Karma Resorts,Other,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,true,Email and Phone,events@karmalakeofmenteith.com,empty


In [21]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Enquiry sent to,Mobile
0,zcrm_3584690000000459415,2019-01-09 11:11:09,empty,Graham,Bovet-White,grahambovet@gmail.com,447831252500,United Kingdom,Karma Resorts,Other,empty,Past Guests,Karma St. Martin's;Karma La Herriza,empty,true,false,true,empty,empty,empty
1,zcrm_3584690000369606560,2022-05-24 12:39:51,empty,Rowan,Johnston,rowan.johnston22962@gmail.com,61427997077,Australia,Karma Resorts,Other,empty,Past Guests,Karma Rottnest;Karma La Herriza,empty,false,false,false,Email And Phone,empty,empty
2,zcrm_3584690000382360020,2022-07-09 22:46:42,2022-07-09,Sijo,Eapen,sijoeapen@gmail.com,919892718274,India,Karma Resorts,Other,Karma Group Booking Request Form,Other,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,false,empty,info@karmagroup.com,empty
3,zcrm_3584690000403535119,2022-10-06 23:33:18,2022-10-06,Josh,Lowsley-Williams,jlowsleywilliams@gmail.com,447779354907,Spain,Karma Resorts,Other,Karma Group Booking Request Form,Other,Karma La Herriza,https://karmagroup.com/find-destination/karma-...,true,true,true,empty,info@karmagroup.com,empty
4,zcrm_3584690000409631627,2022-11-09 18:16:39,empty,Farinaz Neshati,Sani,farinesh@gmail.com,4434928062444,United Kingdom,Karma Resorts,Other,Viewpoint,Past Guests,Karma Royal Candidasa;Karma La Herriza,empty,true,false,true,Email And Phone,empty,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,zcrm_3584690000556728404,2025-06-16 12:33:14,empty,Yanique,-,yaniquesparks@gmail.com,447702059275,United Kingdom,Karma Resorts,Other,Karma La Herriza Database (JUNE 2025) - Weddin...,Web Enquiry,Karma La Herriza,empty,true,true,true,Email and Phone,empty,empty
1097,zcrm_3584690000556728405,2025-06-16 12:33:14,empty,Charlotte,Harris,charlotteharris9@live.co.uk,447979505829,United Kingdom,Karma Resorts,Other,Karma La Herriza Database (JUNE 2025) - Weddin...,Web Enquiry,Karma La Herriza,empty,true,true,true,Email and Phone,empty,empty
1098,zcrm_3584690000556967179,2025-06-15 20:13:24,2025-06-15,empty,Lennart Schmidt,lennart.schmidt@ansys.com,491735211584,Germany,Karma Resorts,Other,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,true,Email and Phone,events@karmalakeofmenteith.com,empty
1099,zcrm_3584690000557133839,2025-06-16 12:15:20,empty,Joseph,Galindo,galindo.joe@gmail.com,17196614592,USA,Karma Resorts,Other,Viewpoint,Past Guests,Karma La Herriza,empty,true,false,false,Email and Phone,empty,empty


In [22]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.csv', index=False)
# find_member_booking.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [23]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [24]:
count_email_only = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty'))].shape[0]
count_phone_only = zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]
count_email_and_phone = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]

count_blank = zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].shape[0]

# zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\blank_data.xlsx', index=False)

zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty')), 'Contact Type'] = 'Email Only'
zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Phone Only'
zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Email and Phone'

zoho_df.drop(zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [25]:
arr_dedup = [count_deleted_wrong_format, count_duplicate_general, count_leads_test, count_email_OTA, count_email_karma, count_blank]
count_sum_dedup = sum(arr_dedup)
count_sum_dedup

26

In [26]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_format))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Lead Do Not Mail : "+str(count_dnm))
# print("Jumlah Setelah Clean Lead Do Not Mail : "+str(count_after_dedup_dnm))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))
print("Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : "+str(count_blank))
print("Jumlah Setelah Wash Phone Email Blank : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))

Summary
Jumlah Filter Leads Awal : 1126
Jumlah Format Email Salah : 0
Jumlah Setelah Clean Format Email Salah : 1126
Jumlah Duplicate (Phone, Mobile dan Email) : 22
Jumlah Setelah Clean Duplicate : 1104
Jumlah Lead Test : 0
Jumlah Setelah Clean Lead Test : 1104
Jumlah Email OTA : 0
Jumlah Setelah Clean Email OTA : 1104
Jumlah Email Karma : 3
Jumlah Setelah Clean Email Karma : 1101
Jumlah Wash Member : 18
Jumlah Wash Booking : 1
Jumlah Setelah Wash Member : 1082
Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : 1
Jumlah Setelah Wash Phone Email Blank : 1081
Contact Type
Jumlah Email Only : 19
Jumlah Phone Only : 25
Jumlah Email dan Phone : 1037


In [27]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Dedup : "+str(count_sum_dedup))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))

Summary Report
Jumlah Data Awal : 1126
Jumlah Wash Dedup : 26
Jumlah Wash Member : 18
Jumlah Wash Booking : 1
Jumlah Data Akhir : 1081


In [28]:
zoho_df.replace('empty', '', inplace=True, regex=True)

zoho_df['First Name'] = zoho_df['First Name'].str.title()
zoho_df['Last Name'] = zoho_df['Last Name'].str.title()

zoho_df.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\Karma La Herriza.csv', index=False)
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\newsletter-clean.xlsx', index=False)